<img src="src/pighat.png" alt="drawing" width="400"/>
<h1 style='text-align:center;'><span style='color:#46aa22;'>Financial Literacy </span><span style='color:#80d162;'>AI </span><span style='color:#46aa22;'>Resource</span></h1>

<a href="https://colab.research.google.com/github/hannahawalsh/FLAIR/blob/main/Long_Form_QA_FLAIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

Write Summary Here

In [ ]:
# Install dependencies
!pip install transformers
!pip install faiss_gpu
!pip install datasets 
!git clone https://github.com/hannahawalsh/FLAIR.git

In [35]:
# Imports
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from datasets import load_dataset
import torch
from tqdm import tqdm
import time
import numpy as np
import re
import os
import faiss
from LFQA_utils import longFormQA, wrap_print, ask_questions

### The Long Form Question Answering Model
A long form question answering model is an AI model that provides long, complex answers to questions. Unlike other question answering models, it does not simply pull snippets of text from a database for its answers, but synthesizes information to create one cohesive answer. It can answer more abstract questions rather than trivia-type ones.  

Our model actually consists of two parts. One is the question-answer model, which pulls passages of text relating to the answer. The second is a sequence to sequence model, which forms those answers into one cohesive answer. 

Some of the functions used in this project are modified from the blogpost and accompanying github code [Explain _Anything_ Like I'm Five: A Model for Open Domain Long Form Question Answering](https://yjernite.github.io/lfqa.html), which goes more in depth on the technical aspects of this type of model. We applied their concepts to fit the needs of our project and apply it to the financial literacy world.  

---

#### Loading the model and the data
Run the below cell to load the model weights and index the financial literacy dataset (if that file doesn't already exist). It might take a little bit if the data hasn't been indexed. You only need to run this cell once per session.

Note: this must be run on a GPU to work. It currently doesn't support running on a CPU. If you're running this in Google Colab, don't worry; it is already set up to run on a small GPU.

In [4]:
# Initialize our Long-Form Question-Answering class
data_filename = "financial_literacy_dataset.csv"
lfqa = longFormQA(data_filename)

# Load model weights from hugging face
qa_model_name = "yjernite/retribert-base-uncased"
s2s_model_name = "yjernite/bart_eli5"
lfqa.load_model_weights(qa_model_name, s2s_model_name)

# Create a dense index of our data if one hasn't been created yet
lfqa.create_dense_index(batch_size=512)

Using custom data configuration default-5ef29c70503a7338
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5ef29c70503a7338/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

### And we're ready to roll!
You can now ask the model questions! Try it out below.

Change how the answers are formed by changing some of the custom parameters.

Do note that because of the limited time and resources of the hackathon, the answers may not always make that much sense or be accurate. It also doesn't know anything about non-financial topics, so don't ask!

In [ ]:
# Ask it a question
question = "What is a credit score?"
answer = lfqa.ask_a_question(Q, **model_kwargs)
wrap_print(answer)

In [ ]:
# Add some custom parameters 
max_question_len = 512
min_answer_len = 256
max_answer_len = 512
question = "What is a credit score?"

answer = lfqa.ask_a_question(Q, **model_kwargs)
wrap_print(answer)

---
Below are a few more examples. Feel free to change the cells to experiment yourself. Note the model is lacking information in some areas so it doesn't answer well, or even correctly. Also watch for how the model parameters affect the answers.

In [40]:
# Here's a few more examples
kwargs = {
    "max_question_len": 1024, 
    "min_answer_len": 64, 
    "max_answer_len": 512} 

questions = [
    "What is the minimum amount I need to open a bank account?",
    "Is there an account for medical costs?",
    "Should I save for retirement if I paid off all my bills?",
    "How do I pay my bills every month?",
    "What's the first thing I should do when I get out of jail?",
    "Are there resources for people leaving prison?"]

ask_questions(questions, kwargs)

Question:
What is a credit score?
Answer:
A credit score is a measure of how likely you are to pay your bills on time. If
you pay your bill on time, you have a higher credit score than if you don't.
Credit scores are used by banks and credit card companies to determine how
likely it is that you will be able to pay them back.

Question:
What is the minimum amount I need to open a bank account?
Answer:
The minimum amount you need to open a bank account is the minimum amount of
money the bank will give you to open an account. If you have more than that,
you'll have to pay a fee to the bank for the privilege of opening an account,
and you'll be charged a fee for every transaction you make.

Question:
Is there an account for medical costs?
Answer:
There is an account for medical costs. It's called a medical expense account.
If you don't have an account, you can't use it for medical expenses. The only
way to use it is if you have a medical emergency, and the insurance company will
pay you fo

In [41]:
# Ask those same questions but with different model parameters
kwargs = {
    "max_question_len": 256, 
    "min_answer_len": 32, 
    "max_answer_len": 256} 
ask_questions(questions, kwargs)

Question:
What is a credit score?
Answer:
A credit score is a measure of how likely you are to pay back your debts on
time. If you pay your bills on time, you have a higher credit score than if you
don't.

Question:
What is the minimum amount I need to open a bank account?
Answer:
There is no minimum amount you need to open a bank account. There is a minimum
amount of money you *need* to open an account. If you do not meet the minimum
amount, you will not be able to open one.

Question:
Is there an account for medical costs?
Answer:
Yes, there is an account for medical costs. It's called a [medical expense
account]( URL_0 ). It's not a bank account, but it's a medical expense account.

Question:
Should I save for retirement if I paid off all my bills?
Answer:
No, you should not. You should save for retirement if you can afford it. If you
can't afford it, you shouldn't be saving for retirement.

Question:
How do I pay my bills every month?
Answer:
You don't have to pay your bills every 

In [42]:
# Try some more questions
questions = [
    "How do I open up a bank account?",
    "What do I need a bank account for?",
    "Why should I open a savings account?",
    "Whats the difference between a savings and checking account?",
    "Whats the difference between credit and debit?",
]
kwargs = {"max_question_len": 256, "min_answer_len": 32, "max_answer_len": 512} 
ask_questions(questions, kwargs)

Question:
How do I open up a bank account?
Answer:
Do you mean how to open up a bank account, or how to get one? If the latter, go
to your bank and ask them. If the former, you'll have to go to a different bank.

Question:
What do I need a bank account for?
Answer:
You don't need a bank account for anything. A bank account is just a way to
transfer money from one place to another. You can use it for a lot of different
things.

Question:
Why should I open a savings account?
Answer:
A savings account is a safe place to store your money for a rainy day. You
don't have to pay taxes on it, and you don't need to worry about the bank taking
your money out of your checking account.

Question:
Whats the difference between a savings and checking account?
Answer:
A checking account is a type of checking account. It's an account that you
can't withdraw money from. A savings account is an account where you can
withdraw money at any time. The difference is that a savings account has a
higher interes

In [43]:
# And those same ones with different parameters
kwargs = {"max_question_len": 256, "min_answer_len": 32, "max_answer_len": 512} 
ask_questions(questions, kwargs)

Question:
How do I open up a bank account?
Answer:
Do you mean how to open up a bank account, or how to get one? If the latter, go
to your bank and ask them. If the former, you'll have to go to a different bank.

Question:
What do I need a bank account for?
Answer:
You don't need a bank account for anything. A bank account is just a way to
transfer money from one place to another. You can use it for a lot of different
things.

Question:
Why should I open a savings account?
Answer:
A savings account is a safe place to store your money for a rainy day. You
don't have to pay taxes on it, and you don't need to worry about the bank taking
your money out of your checking account.

Question:
Whats the difference between a savings and checking account?
Answer:
A checking account is a type of checking account. It's an account that you
can't withdraw money from. A savings account is an account where you can
withdraw money at any time. The difference is that a savings account has a
higher interes

In [45]:
# These answers are using different parameters and it is clear that we 
# need to increase min_answer_len
# When answers are too short, they aren't helpful
kwargs = {"max_question_len": 1024, "min_answer_len": 8, "max_answer_len": 1024} 
questions = [
    "What do I do if I'm poor?",
    "How do I get out of debt?",
    "Can I get a credit card without a credit score?",
    "How do I improve my credit score?",
    "Are there bank accounts for non-citizens?"]
ask_questions(questions, kwargs)

Question:
What do I do if I'm poor?
Answer:
If you're poor, get a job. If you don't have a job, take out a student loan and
pay it off on time.

Question:
How do I get out of debt?
Answer:
You can't get out of debt if you don't pay it off.

Question:
Can I get a credit card without a credit score?
Answer:
Yes, you can get a credit card without a credit score. You just have to prove
that you can pay it off on time.

Question:
How do I improve my credit score?
Answer:
Credit score is a measure of how likely you are to pay your bills on time. If
you don't pay them on time, your score is going to go down.

Question:
Are there bank accounts for non-citizens?
Answer:
Depends on what country you're asking about. If you're talking about a bank
account for a non-citizen, then yes.

